In [1]:
# datar specific

import numpy
from datar import f
from datar.datasets import iris
from datar.base import as_date, factor
from datar.datar import *
from datar.dplyr import mutate, group_by
from datar.tibble import tibble

%run nb_helpers.py
nb_header(
    get, 
    flatten, 
    itemgetter, 
    attrgetter, 
    pd_str, 
    pd_cat, 
    pd_dt, 
    book='datar',
)

### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ get</div>

##### Get a single element or a subset of a dataframe

##### Args:
&emsp;&emsp;`_data`: The dataframe  
&emsp;&emsp;`rows`: The rows to subset the dataframe  
&emsp;&emsp;`cols`: The columns to subset the dataframe  
&emsp;&emsp;&emsp;&emsp;If both rows and cols are scalar, then a single element will be  
&emsp;&emsp;&emsp;&emsp;returned  

##### Returns:
&emsp;&emsp;A single element when both rows and cols are scalar, otherwise  
&emsp;&emsp;a subset of _data  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ flatten</div>

##### Flatten a dataframe into a 1-d python list

##### Args:
&emsp;&emsp;`_data`: The dataframe  

##### Returns:
&emsp;&emsp;The flattened list  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ itemgetter</div>

...

##### Pandas' str accessor for a Series (x.str)

This is helpful when x is a SeriesGroupBy object  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ pd_cat</div>

##### Pandas' cat accessor for a Series (x.cat)

This is helpful when x is a SeriesGroupBy object  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ pd_dt</div>

##### Pandas' dt accessor for a Series (x.dt)

This is helpful when x is a SeriesGroupBy object  


In [2]:
iris >> get(f[:5])

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
,<float64>,<float64>,<float64>,<float64>,<object>
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
iris >> get(cols=f.Species)

,Species
,<object>
0,setosa
1,setosa
2,setosa
3,setosa
...,...
4,setosa
145,virginica
146,virginica
147,virginica


In [4]:
# select single element
iris >> get(1, f.Species)

'setosa'

In [5]:
# get it as a single-element dataframe
iris >> get([1], f.Species)

,Species
,<object>
1,setosa


In [6]:
# or 
iris >> get(1, [f.Species])

,Species
,<object>
1,setosa


In [7]:
df = tibble(x=f[1:3], y=f[3:5])
df >> flatten()

[1, 3, 2, 4]

In [8]:
arr = numpy.array(['a', 'b', 'c', 'd', 'e'])
# df >> mutate(a=arr[f.x], b=arr[f.y])  # Error
df >> mutate(a=itemgetter(arr, f.x), b=itemgetter(arr, f.y))

,x,y,a,b
,<int64>,<int64>,<object>,<object>
0,1,3,b,d
1,2,4,c,e


In [9]:
df = tibble(x=["abc", "def"])
df >> mutate(a=attrgetter(f.x, 'str').upper())

,x,a
,<object>,<object>
0,abc,ABC
1,def,DEF


In [10]:
# or
# df >> mutate(a=pd_str(f.x).upper())
# or
df >> mutate(a=f.x.str.upper())

,x,a
,<object>,<object>
0,abc,ABC
1,def,DEF


In [11]:
# but when df is grouped
gf = df >> group_by(g=[1, 2])
# pd_str(gf.x)[:2].obj
gf >> mutate(a=pd_str(gf.x)[:2])

,x,g,a
,<object>,<int64>,<object>
0,abc,1,ab
1,def,2,de


In [12]:
gf = (
    tibble(x=["2022-01-01", "2022-12-02"])
    >> mutate(x=as_date(f.x, format="%Y-%m-%d"))
    >> group_by(g=[1, 2])
)
gf >> mutate(month=pd_dt(gf.x).month)

,x,g,month
,<datetime64[ns]>,<int64>,<int64>
0,2022-01-01,1,1
1,2022-12-02,2,12


In [13]:
gf = (
    tibble(x=factor([1, 2], levels=[1, 2, 3]))
    >> group_by(g=[1, 2])
)
gf >> mutate(codes=pd_cat(gf.x).codes)

,x,g,codes
,<category>,<int64>,<int8>
0,1,1,0
1,2,2,1
